In [1]:
import numpy as np

In [2]:
def proj(matrix):
    return np.kron(matrix, matrix.conj().T)

In [3]:
qubit0 = np.array([[1], [0]])
qubit1 = np.array([[0], [1]])

qubit = [qubit0, qubit1]

In [4]:
stan0 = np.kron(qubit[0], qubit[0])
stan1 = np.kron(qubit[1], qubit[0])
stan2 = np.kron(qubit[0], qubit[1])

In [10]:
stan0

array([[1],
       [0],
       [0],
       [0]])

In [5]:
stan0teo = np.array([['a0'], ['b0'], ['c0'], ['d0']])
stan1teo = np.array([['a1'], ['b1'], ['c1'], ['d1']])
stan2teo = np.array([['a2'], ['b2'], ['c2'], ['d2']])

In [6]:
baza = [stan0, stan1, stan2]
bazateo = [stan0teo, stan1teo, stan2teo]

In [7]:
check = proj(baza[0]) + proj(baza[1]) + proj(baza[2])
checkteo = proj(bazateo[0]) + proj(bazateo[1]) + proj(bazateo[2])

C:\Users\szymon\AppData\Local\Temp\ipykernel_17504\4267056448.py:2: DeprecationWarning: attempting to conjugate non-numeric dtype; this will error in the future to match the behavior of np.conjugate
  return np.kron(matrix, matrix.conj().T)


UFuncTypeError: ufunc 'multiply' did not contain a loop with signature matching types (dtype('<U2'), dtype('<U2')) -> None

In [9]:
np.kron(qubit[0], qubit[1].conj().T)

array([[0, 1],
       [0, 0]])

In [8]:
np.kron(qubit[1], qubit[0].conj().T)

array([[0, 0],
       [1, 0]])